In [1]:
import pandas as pd
import ast


# Correct Windows path
path = r"D:\vc-research\vc-research\Reese's contributions\reese data\downloaded_batches\fullCOISample.csv"

df = pd.read_csv(path)



#remove all description columns and the liquidation preference column (it is in the liquidation preference amount column)
# Suffixes to drop
suffixes = ("_d", "_liquidation_preference", ".description")

# Drop all columns ending with any of the suffixes
df = df.drop(columns=[col for col in df.columns if col.endswith(suffixes)])


def count_series(x):
    if isinstance(x, list):
        return len(x)
    if isinstance(x, str):
        x_clean = x.strip().upper()
        if x_clean in ("MISSING", "UNCERTAIN", ""):
            return 0
        try:
            parsed = ast.literal_eval(x)   # safely interpret string '[...]'
            if isinstance(parsed, list):
                return len(parsed)
        except:
            return 0
    return 0

df["number_of_series"] = df["preferred_stock_series"].apply(count_series)

def count_true(row):
    # Count both string "TRUE" and boolean True
    return sum(
        (val is True) or
        (isinstance(val, str) and val.strip().upper() == "TRUE")
        for val in row
    )

df["true_count"] = df.apply(count_true, axis=1)

df["series_inaccuracy"] = df["true_count"] - df["number_of_series"]

summary_stats = df["series_inaccuracy"].describe()
print("Summary statistics for series_inaccuracy:")
print(summary_stats)
# Convert filing_date to datetime, ignoring bad values like "MISSING"
df["filing_date"] = pd.to_datetime(df["filing_date"], errors="coerce")


# Sort by company and date
df = df.sort_values(["company_id", "filing_date"])

# For each company ID, mark first and last
df["firstlast"] = df.groupby("company_id")["filing_date"] \
                    .transform(lambda x: ["first"] + ["middle"]*(len(x)-2) + ["last"] if len(x) > 1 else ["only"])

# Keep only first and last observations (drop middles)
df = df[df["firstlast"].isin(["first", "last", "only"])]

# Reset index for cleanliness
df = df.reset_index(drop=True)

# Print dtypes of remaining columns
print(df.dtypes)

C:\Users\Owner\AppData\Local\Temp\ipykernel_61588\829999650.py:8: DtypeWarning: Columns (7,9,13,15,18,19,20,21,22,23,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,27

Summary statistics for series_inaccuracy:
count    28616.000000
mean        -1.083904
std          2.350946
min        -28.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          2.000000
Name: series_inaccuracy, dtype: float64
custom_id                                                    object
company_id                                                    int64
company_name                                                 object
document_title                                               object
inc_document                                                  int64
                                                              ...  
preferred_stock_terms_Series C*_special_class_protections    object
number_of_series                                              int64
true_count                                                    int64
series_inaccuracy                                             int64
firstlast                                                    o

In [2]:
df.to_csv(r"D:\vc-research\vc-research\Reese's contributions\Reese postprocessing outputs\firstlastCOISample.csv", index=False)

In [3]:
# Summary stats by firstlast group
summary = df.groupby("firstlast")["number_of_series"].describe()

print(summary)


            count      mean       std  min  25%  50%  75%   max
firstlast                                                      
first      4297.0  1.493600  1.832618  0.0  0.0  1.0  2.0  15.0
last       4297.0  3.527577  2.694785  0.0  2.0  3.0  5.0  24.0
only       3208.0  2.844763  1.986444  0.0  1.0  2.0  4.0  16.0


In [4]:
import pandas as pd

def reshape_preferred_stock_terms(df):
    # Metadata columns to carry over
    meta_cols = [
        "custom_id",
        "company_id",
        "company_name",
        "document_title",
        "filing_date",
        "inc_document",
    ]
    
    # 1. Identify all series from the *_present columns
    present_cols = [
        c for c in df.columns
        if c.startswith("preferred_stock_terms_") and c.endswith("_present")
    ]
    
    # Extract the series name between the prefix and "_present"
    prefix = "preferred_stock_terms_"
    suffix = "_present"
    series_list = [
        c[len(prefix):-len(suffix)]
        for c in present_cols
    ]
    
    # 2. Precompute all columns for each series
    series_to_cols = {}
    for series in series_list:
        series_prefix = f"{prefix}{series}_"
        # all columns for this series
        cols = [c for c in df.columns if c.startswith(series_prefix)]
        series_to_cols[series] = cols
    
    # 3. Build the output rows
    out_rows = []
    
    for _, row in df.iterrows():
        meta = row[meta_cols].to_dict()
        
        for series in series_list:
            present_col = f"{prefix}{series}{suffix}"
            if bool(row.get(present_col)):
                series_cols = series_to_cols[series]
                
                # Build dict of series-specific values, stripping the prefix from the column names
                series_data = {}
                series_prefix = f"{prefix}{series}_"
                for col in series_cols:
                    if col == present_col:
                        continue  # don't keep the _present flag as a variable
                    var_name = col[len(series_prefix):]  # part after preferred_stock_terms_[series]_
                    series_data[var_name] = row[col]
                
                new_row = {
                    **meta,
                    "series": series,
                    **series_data
                }
                out_rows.append(new_row)
    
    # 4. Create the final dataframe: one row per (company, series)
    result = pd.DataFrame(out_rows)
    
    # Optional: order columns with series first, then meta, then variables
    variable_cols = [c for c in result.columns if c not in ["series", *meta_cols]]
    result = result[["series", *meta_cols, *variable_cols]]
    
    return result

# Usage:
# new_df = reshape_preferred_stock_terms(df)
# Correct Windows path
path = r"D:\vc-research\vc-research\Reese's contributions\reese data\downloaded_batches\fullCOISample.csv"
df = pd.read_csv(path)
series_df = reshape_preferred_stock_terms(df)

C:\Users\Owner\AppData\Local\Temp\ipykernel_61588\1065439639.py:76: DtypeWarning: Columns (7,9,13,15,18,19,20,21,22,23,35,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,

In [5]:
import numpy as np
import pandas as pd
import re

def has_too_many_letters(x):
    """
    Returns True if a string contains more than 2 alphabetic characters.
    """
    if isinstance(x, str):
        letters = re.findall(r"[A-Za-z]", x)
        return len(letters) > 2
    return False


def to_numeric_safe(x):
    """
    Convert a value to float after stripping non-numeric characters.
    Returns np.nan if conversion fails.
    """
    if isinstance(x, (int, float, np.number)):
        return float(x)

    if isinstance(x, str):
        cleaned = re.sub(r"[^0-9\.\-]", "", x)
        if cleaned in ("", ".", "-", None):
            return np.nan
        try:
            return float(cleaned)
        except ValueError:
            return np.nan

    return np.nan


def compute_liquidation_minimal(row, shares_col="shares_authorized"):
    raw_shares = row.get(shares_col, np.nan)
    raw_pref = row.get("liquidation_preference.amount", np.nan)

    # --- NEW RULE: Too many letters triggers error ---
    if has_too_many_letters(raw_shares) or has_too_many_letters(raw_pref):
        return pd.Series({
            "liquidation_amount": np.nan,
            "liquidation_error": 1
        })

    shares = to_numeric_safe(raw_shares)
    pref = to_numeric_safe(raw_pref)

    valid_shares = pd.notnull(shares)
    valid_pref = pd.notnull(pref)

    if not valid_shares or not valid_pref:
        print(
            f"Liquidation error: shares raw=({raw_shares}, type={type(raw_shares)}), "
            f"converted={shares}; pref raw=({raw_pref}, type={type(raw_pref)}), converted={pref}"
        )
        return pd.Series({
            "liquidation_amount": np.nan,
            "liquidation_error": 1
        })

    # Compute liquidation amount
    liquidation_amount = shares * pref

    return pd.Series({
        "liquidation_amount": liquidation_amount,
        "liquidation_error": 0
    })


# ---- Apply to dataframe ----
df_liq = series_df.apply(compute_liquidation_minimal, axis=1)
series_df = pd.concat([series_df, df_liq], axis=1)

# ---- Split into error / no-error dataframes ----
df_liq_error = series_df[series_df["liquidation_error"] == 1].copy()
df_liq_noerror = series_df[series_df["liquidation_error"] == 0].copy()


Liquidation error: shares raw=(2662500, type=<class 'str'>), converted=2662500.0; pref raw=(nan, type=<class 'float'>), converted=nan
Liquidation error: shares raw=(5100000, type=<class 'str'>), converted=5100000.0; pref raw=(nan, type=<class 'float'>), converted=nan
Liquidation error: shares raw=(2562500, type=<class 'str'>), converted=2562500.0; pref raw=(nan, type=<class 'float'>), converted=nan
Liquidation error: shares raw=(7166668, type=<class 'str'>), converted=7166668.0; pref raw=(nan, type=<class 'float'>), converted=nan
Liquidation error: shares raw=(9525000, type=<class 'str'>), converted=9525000.0; pref raw=(nan, type=<class 'float'>), converted=nan
Liquidation error: shares raw=(9600000, type=<class 'str'>), converted=9600000.0; pref raw=(nan, type=<class 'float'>), converted=nan
Liquidation error: shares raw=(9819635, type=<class 'str'>), converted=9819635.0; pref raw=(nan, type=<class 'float'>), converted=nan
Liquidation error: shares raw=(13132438, type=<class 'str'>), 

In [7]:
series_df.to_csv(r"D:\vc-research\vc-research\Reese's contributions\Reese postprocessing outputs\preferred_stock_series_expanded.csv", index=False)

df_liq_error.to_csv(r"D:\vc-research\vc-research\Reese's contributions\Reese postprocessing outputs\preferred_stock_series_liq_errors.csv", index=False)
df_liq_noerror.to_csv(r"D:\vc-research\vc-research\Reese's contributions\Reese postprocessing outputs\preferred_stock_series_liq_no_errors.csv", index=False)

[Open preferred stock expanded CSV](file:///D:/vc-research/vc-research/Reese%27s%20contributions/Reese%20postprocessing%20outputs/preferred_stock_series_expanded.csv)


In [ ]:
# -----------------------------
# 1) Summary stats for liquidation_amount
# -----------------------------

# Exclude NaNs from summary stats
liq_summary = series_df["liquidation_amount"].dropna().describe()

print(liq_summary)  # or keep as a series/dataframe if you prefer

# If you want it as a 1-row dataframe:
liq_summary_df = liq_summary.to_frame().T

count    3.487800e+04
mean     3.888123e+11
std      4.608685e+13
min      0.000000e+00
25%      2.423837e+06
50%      6.548863e+06
75%      1.518000e+07
max      8.288322e+15
Name: liquidation_amount, dtype: float64


In [8]:
# Make sure filing_date is a datetime if you want proper grouping/sorting
# (optional but recommended)
# df["filing_date"] = pd.to_datetime(df["filing_date"], errors="coerce")

group_cols = ["company_id", "filing_date"]

company_date_liq = (
    series_df.groupby(group_cols, as_index=False)
      .agg(
          total_liquidation_amount=("liquidation_amount", "sum"),
          custom_id=("custom_id", "first"),
          company_name=("company_name", "first"),
          document_title=("document_title", "first"),
          inc_document=("inc_document", "first"),
      )
)

# Reorder columns to match your meta_cols + new total column
company_date_liq = company_date_liq[
    [
        "custom_id",
        "company_id",
        "company_name",
        "document_title",
        "filing_date",
        "inc_document",
        "total_liquidation_amount",
    ]
]


In [9]:
company_date_liq.to_csv(r"D:\vc-research\vc-research\Reese's contributions\Reese postprocessing outputs\company_date_total_liquidation.csv", index=False)